In [1]:
pip install pandas sqlalchemy psycopg2 pyarrow


  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached SQLAlchemy-2.0.37-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached numpy-2.2.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached greenlet-3.1.1-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached SQLAlchemy-2.0.37-cp312-cp312-win_amd64.whl (2.1 MB)
Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl (1.2 MB)
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ------- -------------------------------- 5.0/25.2 MB 25.2 MB/s eta 0:00:01
   ------------------------------------- -- 23.9/25.2 MB 60.5 MB/s eta 0:00:01
   --

In [11]:
import argparse, os, sys
from time import time
import pandas as pd 
import pyarrow.parquet as pq
from sqlalchemy import create_engine

In [46]:
user = "root"
password = "root"
host = "pgdatabase"
port = "5432"
db = "ny_taxi"
tb_main = "green_tripdata"
tb_lookup = "taxi_zone"
url_main = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"
url_lookup = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"
save_path = "./data/"


In [50]:
file_name = url_main.rsplit('/', 1)[-1].strip()
full_path = os.path.join(save_path, file_name)
os.system(f"curl -L {url_main} -o {full_path}")

0

In [96]:
df = pd.read_csv(full_path,index_col=False)


C:\Users\topta\AppData\Local\Temp\ipykernel_26644\1351154296.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_path,index_col=False)


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               387007 non-null  float64
 1   lpep_pickup_datetime   476386 non-null  object 
 2   lpep_dropoff_datetime  476386 non-null  object 
 3   store_and_fwd_flag     387007 non-null  object 
 4   RatecodeID             387007 non-null  float64
 5   PULocationID           476386 non-null  int64  
 6   DOLocationID           476386 non-null  int64  
 7   passenger_count        387007 non-null  float64
 8   trip_distance          476386 non-null  float64
 9   fare_amount            476386 non-null  float64
 10  extra                  476386 non-null  float64
 11  mta_tax                476386 non-null  float64
 12  tip_amount             476386 non-null  float64
 13  tolls_amount           476386 non-null  float64
 14  ehail_fee              0 non-null   

In [82]:
object_columns = df.select_dtypes(include=["object"]).columns

for col in object_columns:
    unique_types = df[col].apply(type).unique()
    print(f"Unique types in {col}: {unique_types}")


Unique types in lpep_pickup_datetime: [<class 'str'>]
Unique types in lpep_dropoff_datetime: [<class 'str'>]
Unique types in store_and_fwd_flag: [<class 'str'> <class 'float'>]


In [100]:
str_values = df[df["store_and_fwd_flag"].apply(lambda x: isinstance(x, str))]
str_values["store_and_fwd_flag"].unique()

array(['N', 'Y'], dtype=object)

In [101]:
float_values = df[df["store_and_fwd_flag"].apply(lambda x: isinstance(x, float))]
float_values['store_and_fwd_flag'].unique()

array([nan], dtype=object)

In [89]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [ ]:
df_iter = pd.read_csv(full_path,index_col=False,terator=True, chunksize=100000)
while True: 
    try:
        t_start = time()

        if file_name == "green_tripdata_2019-10.csv.gz":
            df = next(df_iter)
            df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
            df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])
            df.to_sql(name=tb_main, con=engine, if_exists='replace')
        elif file_name == "taxi_zone_lookup.csv":
            df = next(df_iter)
            file_name == "green_tripdata_2019-10.csv.gz"
            df.to_sql(name=tb_lookup, con=engine, if_exists='replace')

        t_end = time()

        print('inserted another chunk, took %.3f second' % (t_end - t_start))

    except StopIteration:
        print("Finished ingesting data into the postgres database")
        break


df.to_sql(name=tb_main,con=engine,if_exists='append')

Rows where 'store_and_fwd_flag' is string:


array(['N', 'Y'], dtype=object)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               387007 non-null  float64
 1   lpep_pickup_datetime   476386 non-null  object 
 2   lpep_dropoff_datetime  476386 non-null  object 
 3   store_and_fwd_flag     387007 non-null  object 
 4   RatecodeID             387007 non-null  float64
 5   PULocationID           476386 non-null  int64  
 6   DOLocationID           476386 non-null  int64  
 7   passenger_count        387007 non-null  float64
 8   trip_distance          476386 non-null  float64
 9   fare_amount            476386 non-null  float64
 10  extra                  476386 non-null  float64
 11  mta_tax                476386 non-null  float64
 12  tip_amount             476386 non-null  float64
 13  tolls_amount           476386 non-null  float64
 14  ehail_fee              0 non-null   

In [ ]:
print('\n')

# Create SQL engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

file = pq.ParquetFile(file_name)
df = next(file.iter_batches(batch_size=10)).to_pandas()
df_iter = file.iter_batches(batch_size=100000)

# Create the table
df.head(0).to_sql(name=tb, con=engine, if_exists='replace')

# Insert values
t_start = time()
count = 0
for batch in df_iter:
    count+=1

    batch_df = batch.to_pandas()

    print(f'inserting batch {count}...')

    b_start = time()
    batch_df.to_sql(name=tb, con=engine, if_exists='append')
    b_end = time()

    print(f'inserted! time taken {b_end-b_start:10.3f} seconds.\n')
    
t_end = time()   
print(f'Completed! Total time taken was {t_end-t_start:10.3f} seconds for {count} batches.')    